In [8]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

In [ ]:
from ultralytics import YOLO

In [ ]:
Directory = r"F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1"
FileName = "2024_07_09_11_30_174.jpg"
raw_img = cv.imread(Directory + "/" + FileName)
img = cv.cvtColor(raw_img, cv.COLOR_BGR2GRAY)
img = raw_img

In [2]:
model = YOLO('best.pt', task='segment')

In [ ]:
model = YOLO('best.pt')
results = model(img, conf=0.3, save=True, show_labels=False, retina_masks=True)

**Pre and Post processing time**
- reduced from 5 seconds to ms by resizing the image before applying to the model

# Video Inferencing

In [ ]:
from ultralytics.utils.plotting import Annotator, colors

In [49]:
def im_show(img):
    cv.imshow('1', img)
    cv.waitKey()

In [106]:
# Camera Perspective View to Top View Transformation
def camera_angle2top_view():
    points = pd.read_csv('Points.csv')
    ref_pts = np.array([points.iloc[i,:2] for i in range(1, len(points))], dtype = 'uint16')
    dst_pts = np.array([points.iloc[i,2:] for i in range(1, len(points))], dtype = 'uint16')
    h, status = cv.findHomography(ref_pts, dst_pts)
    #print(ref_pts, dst_pts)
    return h

In [36]:
# def InferenceImage(frame, model, width, height, alpha, im_layout, h):
#     img_resize = cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
#     results = model(img_resize, conf=0.5)
#     if results[0].masks is not None:
#         mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
#         mask_img_3 = np.stack([mask_img, np.zeros(mask_img.shape), np.zeros(mask_img.shape)], axis=2)
#         temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
#         im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
#         layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
# #         mask_inv = np.logical_not(np.stack([mask_img, mask_img, mask_img], axis=2))
# #         temp_image = np.zeros(orig_img.shape)
# #         temp_image = mask_inv * orig_img
# #         temp_image += (mask_img_3 * 255).astype('uint8')
#     else:
#         temp_image = img_resize
#         layout_mask = im_layout
#     return temp_image, layout_mask

In [142]:
count_frame = 0
fps = 1
alpha = 0.8
Directory = r"F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1"

width = height = 640
Files = os.listdir(Directory)
Files = ['2024_07_09_11_30_114.jpg', '2024_07_09_11_30_115.jpg', '2024_07_09_11_30_116.jpg', '2024_07_09_11_30_117.jpg', '2024_07_09_11_30_118.jpg']
im_ref = cv.imread('ReferenceImage.jpg')
im_layout = cv.imread('SquareLayout.jpg')    

In [100]:
model = YOLO('best.pt')

In [143]:
out = cv.VideoWriter("inst_segment.avi", cv.VideoWriter_fourcc(*"MJPG"), fps, (3*width, height))
#out2 = cv.VideoWriter("inst_seg_layout.avi", cv.VideoWriter_fourcc(*"MJPG"), fps, (im_layout.shape[1], im_layout.shape[0]))
video = cv.VideoCapture(r"F:\Work\SOCAAutomation\Dataset\NA_DISOPSAL_SOCA_2024\09072024\AXIS 213 - 10.1.5.173 2024-07-09_11_30_00_000.asf")
dur = [[3,0], [3,45]] #[mins,secs]
startFrame = (dur[0][0] * 60 + dur[0][1] ) * 15
endFrame = (dur[1][0] * 60 + dur[1][1] ) * 15

In [144]:
count_frame = 0
h = camera_angle2top_view()
while True:
    ret, frame = video.read()
    if ret:
        count_frame += 1
        if count_frame > startFrame and count_frame < endFrame:
            #frame = cv.imread(Directory + "/" + Files[count_frame])
            img_resize = cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
            results = model(img_resize, conf=0.5)
            if results[0].masks is not None:
                mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
                mask_img_3 = np.stack([np.zeros(mask_img.shape), mask_img, np.zeros(mask_img.shape)], axis=2)
                temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
                im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))
                im_layout_inv = 255 - im_layout
                layout_mask = cv.add(im_layout_inv, (im_out * 255).astype('uint8'))
                #layout_mask = cv.addWeighted(im_layout, 0.3 , (im_out * 255).astype('uint8'), 1-0.3, 0)
            else:
                temp_image = img_resize
                layout_mask = 255 - im_layout
                
            out.write(np.hstack([img_resize, temp_image, layout_mask]))
            #out2.write(layout_mask)        
    else:
        break
out.release()
#out2.release()
video.release()


0: 640x640 1 Fire-Smoke-Sodium-Tray, 3615.2ms
Speed: 6.0ms preprocess, 3615.2ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4861.4ms
Speed: 7.0ms preprocess, 4861.4ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5109.9ms
Speed: 6.0ms preprocess, 5109.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4075.5ms
Speed: 7.6ms preprocess, 4075.5ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4891.3ms
Speed: 15.5ms preprocess, 4891.3ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4923.4ms
Speed: 6.0ms preprocess, 4923.4ms inference, 19.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 5982.0ms
Speed: 5.0ms preprocess, 5982.0ms inference, 610.1ms postprocess per imag

Speed: 10.0ms preprocess, 4408.5ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4127.7ms
Speed: 7.0ms preprocess, 4127.7ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 3968.6ms
Speed: 6.0ms preprocess, 3968.6ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4019.6ms
Speed: 17.0ms preprocess, 4019.6ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 3929.5ms
Speed: 7.1ms preprocess, 3929.5ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4580.1ms
Speed: 7.0ms preprocess, 4580.1ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4029.6ms
Speed: 7.0ms preprocess, 4029.6ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smok


0: 640x640 4 Fire-Smoke-Sodium-Trays, 5237.2ms
Speed: 9.0ms preprocess, 5237.2ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 5297.8ms
Speed: 8.0ms preprocess, 5297.8ms inference, 32.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 5176.7ms
Speed: 8.5ms preprocess, 5176.7ms inference, 27.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 5034.1ms
Speed: 9.0ms preprocess, 5034.1ms inference, 23.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 5721.3ms
Speed: 13.5ms preprocess, 5721.3ms inference, 16.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 Fire-Smoke-Sodium-Trays, 5451.9ms
Speed: 6.5ms preprocess, 5451.9ms inference, 28.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Fire-Smoke-Sodium-Trays, 5403.9ms
Speed: 8.0ms preprocess, 5403.9ms inference, 30.5ms postprocess

0: 640x640 4 Fire-Smoke-Sodium-Trays, 5655.1ms
Speed: 10.0ms preprocess, 5655.1ms inference, 19.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 5300.8ms
Speed: 8.5ms preprocess, 5300.8ms inference, 16.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 Fire-Smoke-Sodium-Trays, 5100.6ms
Speed: 8.0ms preprocess, 5100.6ms inference, 28.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 5192.7ms
Speed: 6.0ms preprocess, 5192.7ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 5622.6ms
Speed: 11.5ms preprocess, 5622.6ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 Fire-Smoke-Sodium-Trays, 5107.1ms
Speed: 8.0ms preprocess, 5107.1ms inference, 18.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 Fire-Smoke-Sodium-Trays, 5898.3ms
Speed: 12.0ms preprocess, 5898.3ms inference, 22.5ms postproces

Speed: 7.0ms preprocess, 3722.3ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 3748.4ms
Speed: 6.0ms preprocess, 3748.4ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 3762.4ms
Speed: 5.0ms preprocess, 3762.4ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 3785.5ms
Speed: 6.0ms preprocess, 3785.5ms inference, 20.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 3638.3ms
Speed: 7.0ms preprocess, 3638.3ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fire-Smoke-Sodium-Trays, 3686.3ms
Speed: 6.0ms preprocess, 3686.3ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4231.8ms
Speed: 7.1ms preprocess, 4231.8ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 Fir

...


0: 640x640 1 Fire-Smoke-Sodium-Tray, 3930.5ms
Speed: 7.1ms preprocess, 3930.5ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 12 Fires, 3724.3ms
Speed: 7.1ms preprocess, 3724.3ms inference, 44.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4015.6ms
Speed: 13.0ms preprocess, 4015.6ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4038.6ms
Speed: 6.0ms preprocess, 4038.6ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 3859.5ms
Speed: 6.0ms preprocess, 3859.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 3715.3ms
Speed: 7.1ms preprocess, 3715.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4371.0ms
Speed: 7.0ms preprocess, 4371.0ms inference, 6.0ms postprocess per image at shape (1, 3, 64


0: 640x640 (no detections), 6650.8ms
Speed: 8.0ms preprocess, 6650.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6418.3ms
Speed: 28.0ms preprocess, 6418.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5553.5ms
Speed: 9.0ms preprocess, 5553.5ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5535.0ms
Speed: 5.5ms preprocess, 5535.0ms inference, 10.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5411.4ms
Speed: 8.0ms preprocess, 5411.4ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5144.6ms
Speed: 10.0ms preprocess, 5144.6ms inference, 11.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 6376.4ms
Speed: 14.0ms preprocess, 6376.4ms inference, 10.0ms postprocess per image at shape (1, 3, 

Speed: 7.0ms preprocess, 5361.8ms inference, 10.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5328.3ms
Speed: 11.5ms preprocess, 5328.3ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5030.2ms
Speed: 10.0ms preprocess, 5030.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5109.6ms
Speed: 8.5ms preprocess, 5109.6ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5803.3ms
Speed: 11.5ms preprocess, 5803.3ms inference, 13.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5530.0ms
Speed: 13.0ms preprocess, 5530.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5205.2ms
Speed: 13.0ms preprocess, 5205.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 5322.8m


0: 640x640 2 Fire-Smoke-Sodium-Trays, 4051.7ms
Speed: 9.0ms preprocess, 4051.7ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 3981.6ms
Speed: 7.0ms preprocess, 3981.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 3988.6ms
Speed: 6.0ms preprocess, 3988.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4137.7ms
Speed: 6.0ms preprocess, 4137.7ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 4364.0ms
Speed: 6.0ms preprocess, 4364.0ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 4146.8ms
Speed: 6.0ms preprocess, 4146.8ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Fire-Smoke-Sodium-Tray, 4230.8ms
Speed: 7.0ms preprocess, 4230.8ms inference, 8.0ms postprocess per im


0: 640x640 (no detections), 4563.6ms
Speed: 7.9ms preprocess, 4563.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 4550.6ms
Speed: 0.0ms preprocess, 4550.6ms inference, 20.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Fire-Smoke-Sodium-Trays, 4580.2ms
Speed: 0.0ms preprocess, 4580.2ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 4240.2ms
Speed: 10.0ms preprocess, 4240.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 3950.2ms
Speed: 10.0ms preprocess, 3950.2ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 4100.2ms
Speed: 10.0ms preprocess, 4100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 3900.2ms
Speed: 10.0ms preprocess, 3900.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (n

In [136]:
mask_img_3 = np.stack([np.zeros(mask_img.shape), mask_img, np.zeros(mask_img.shape)], axis=2)
temp_image = cv.addWeighted(img_resize, alpha , (mask_img_3 * 255).astype('uint8'), 1-alpha, 0)        
im_out = cv.warpPerspective(mask_img_3, h, (im_layout.shape[1],im_layout.shape[0]))

im_layout_inv = 255 - im_layout
layout_mask = cv.add(im_layout_inv, (im_out * 255).astype('uint8'))
im_show(np.hstack([temp_image, im_layout_inv, layout_mask]))


In [112]:
out.release()
out2.release()
video.release()

In [ ]:
alpha = 0.8
dst = cv.addWeighted(im1, alpha , im2, 1-alpha, 0) 
plt.imshow(im2)

In [ ]:
alpha = 0.6
im1 = cv.imread(r'F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1\2024_07_09_11_30_32.jpg')
im1 = cv.resize(im1, (640, 640), interpolation = cv.INTER_LINEAR)
#results = model(im1, conf=0.5) #, save=True, show_labels=False, retina_masks=True)
mask_img = np.any(np.array(results[0].masks[:].data), axis=0)
orig_img = im1 #results[0].orig_img
mask_img_3 = np.stack([mask_img, np.zeros(mask_img.shape), np.zeros(mask_img.shape)], axis=2)

im2 = np.array(mask_img_3 * 255, dtype = 'uint8')

dst = cv.addWeighted(im1, alpha , im2, 1-alpha, 0) 
# mask_inv = np.logical_not(np.stack([mask_img, mask_img, mask_img], axis=2))
# temp_image = np.zeros(orig_img.shape)
# temp_image = mask_inv * orig_img
# temp_image += (mask_img_3 * 255).astype('uint8')


In [ ]:
plt.imshow(mask_img_3)

In [ ]:
plt.imshow(dst)

In [ ]:
frame = cv.imread(r'F:\Work\SOCAAutomation\Dataset\Images\09072024_1_1\2024_07_09_11_30_344.jpg')
img_resize = cv.resize(frame, (640, 640), interpolation = cv.INTER_LINEAR)
cv.imwrite('ReferenceImage.jpg', img_resize)

# Transform Model to Onnx

In [ ]:
model.export(format="onnx")  # creates 'yolov8n_openvino_model/

In [ ]:
onnx_model = YOLO("best.onnx", task='segment')

In [ ]:
res_onnx = onnx_model(img, conf=0.3)

## ONNX Model Optimizations

### F32 to F16

In [ ]:
import onnx
from onnxconverter_common import float16

In [ ]:
onnx_model = onnx.load("best.onnx")
onnx_model_fp16 = float16.convert_float_to_float16(onnx_model)
onnx.save(onnx_model_fp16, "model_fp16.onnx")

In [ ]:
onnx_model = YOLO("model_fp16.onnx", task='segment')

In [ ]:
res_onnx_fp16 = onnx_model(img, conf=0.3)

In [ ]:
img.dtype

# Transform Model to OpenVino

In [ ]:
model.export(format="openvino")  # creates 'yolov8n_openvino_model/

In [ ]:
ov_model = YOLO("yolov8n_openvino_model/")

# Run inference
results = ov_model("https://ultralytics.com/images/bus.jpg")

In [88]:
(15 / 240 ) * 640

40.0